#  __Deep Learning Models__



## Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D, Conv1D, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [ ]:
#Load the cleaned dataset
df = pd.read_csv(r'C:\group-1-main\Model-Evaluvation\cleaned_data.csv')
df['tweet'] = df['tweet'].astype(str)


## Encoding and Handling Data Imbalance

In [ ]:
# Split the data into training and testing sets
X = df['tweet']
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the tokenizer on training data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Transform training and test data
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to the same length
X_train_pad = pad_sequences(X_train_seq, maxlen=100)
X_test_pad = pad_sequences(X_test_seq, maxlen=100)

vocab_size = len(tokenizer.word_index) + 1

# Address class imbalance using SMOTE on training data
smote = SMOTE(random_state=42)
X_train_pad_res, y_train_pad_res = smote.fit_resample(X_train_pad, y_train)


# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

## 1.  __LSTM__ model

In [ ]:
# Define LSTM model
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=vocab_size, output_dim=128))
lstm_model.add(SpatialDropout1D(0.2))  # Dropout layer to prevent overfitting
lstm_model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01))))
lstm_model.add(Dense(3, activation='softmax'))  # Assuming 3 classes

# Compile the model with Adam optimizer
lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the LSTM model with early stopping
lstm_model.fit(X_train_pad_res, y_train_pad_res, epochs=20, batch_size=64, validation_data=(X_test_pad, y_test), callbacks=[early_stopping])

# Evaluate LSTM model
lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test_pad, y_test)
lstm_y_pred = lstm_model.predict(X_test_pad)
lstm_y_pred_classes = np.argmax(lstm_y_pred, axis=1)
lstm_report = classification_report(y_test, lstm_y_pred_classes, target_names=['Normal', 'Hate', 'Offensive'])

print("LSTM Model Accuracy:", lstm_accuracy)
print("LSTM Model Classification Report:\n", lstm_report)

## 2.  __CNN__ 

In [ ]:
# Define CNN model
cnn_model = Sequential()
cnn_model.add(Embedding(input_dim=vocab_size, output_dim=128))
cnn_model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(3, activation='softmax'))  # Assuming 3 classes

# Compile the model with Adam optimizer
cnn_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the CNN model with early stopping
cnn_model.fit(X_train_pad_res, y_train_pad_res, epochs=20, batch_size=64, validation_data=(X_test_pad, y_test), callbacks=[early_stopping])

# Evaluate CNN model
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test_pad, y_test)
cnn_y_pred = cnn_model.predict(X_test_pad)
cnn_y_pred_classes = np.argmax(cnn_y_pred, axis=1)
cnn_report = classification_report(y_test, cnn_y_pred_classes, target_names=['Normal', 'Hate', 'Offensive'])

print("CNN Model Accuracy:", cnn_accuracy)
print("CNN Model Classification Report:\n", cnn_report)

## 3.  __Bidirectional LSTM__ model

In [ ]:
# Define Bidirectional LSTM model
bi_lstm_model = Sequential()
bi_lstm_model.add(Embedding(input_dim=vocab_size, output_dim=128))
bi_lstm_model.add(SpatialDropout1D(0.2))  # Dropout layer to prevent overfitting
bi_lstm_model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01))))
bi_lstm_model.add(Dense(3, activation='softmax'))  # Assuming 3 classes

# Compile the model with Adam optimizer
bi_lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the Bidirectional LSTM model with early stopping
bi_lstm_model.fit(X_train_pad_res, y_train_pad_res, epochs=20, batch_size=64, validation_data=(X_test_pad, y_test), callbacks=[early_stopping])

# Evaluate Bidirectional LSTM model
bi_lstm_loss, bi_lstm_accuracy = bi_lstm_model.evaluate(X_test_pad, y_test)
bi_lstm_y_pred = bi_lstm_model.predict(X_test_pad)
bi_lstm_y_pred_classes = np.argmax(bi_lstm_y_pred, axis=1)
bi_lstm_report = classification_report(y_test, bi_lstm_y_pred_classes, target_names=['Normal', 'Hate', 'Offensive'])

print("Bidirectional LSTM Model Accuracy:", bi_lstm_accuracy)
print("Bidirectional LSTM Model Classification Report:\n", bi_lstm_report)